# LangChain
LangChain is a framework for developing applications powered by language models.

## Schema
Most interfaces in LangChain are text based because most language models support only "text in, text out" (if you are interested, we can have an hour long discussion where I can explain you everything about transformers architecture).

### How LLMs work (practical approach, not theory)
LLMs have 3 parts in prompt usually.
1. System Chain Message
    - A chat message representing information that should be instructions to the AI system.
2. Human Chat Message
    - A chat message representing information coming from a human interacting with the AI system.
3. AI Chat Message
    - A chat message representing information coming from the AI system.

**__PRO_TIP__**: While making product, try to keep Human chat as low as possible because normal users are very bad at writing prompt and model give pretty bad results with bad prompts which will results into you replying to useless complaint emails.

Other than that, one important thing I need to explain.
>    - Our daily usage is when we use model with zero-shot learning but model give outstanding result when it is fine-tuned with some examples, usually known as few-shot learning. We use method of `Examples` for this where we provide model with input/output pairs so model can better understand what we are expecting. 

While talking about terminology, we use word `Document` for unstructured data. Documents have compeletely random formats so normal programming approach cannot get desired data from doc. That is why we use LLMs because they can really understand data and give response accordingly. Document has two parts, `page_content` and `metadata`.

## Models
LangChain deals with 3 types of models.
1. Language Models
2. Chat Models
3. Text Embedding Models

LLMs: <br>
These models take a text string as input, and return a text string as output.<br><br>
Chat Models:<br>
These models are usually backed by a language model, but their APIs are more structured. Specifically, these models take a list of Chat Messages as input, and return a Chat Message.<br><br>
Text Embedding Models<br>
These models take text as input and return a list of floats.<br>

### LLMs
Large Language Models (LLMs) are a core component of LangChain. LangChain is not a provider of LLMs, but rather provides a standard interface through which you can interact with a variety of LLMs. The LLM class is a class designed for interfacing with LLMs. There are lots of LLM providers (OpenAI, Cohere, Hugging Face, etc) - this class is designed to provide a standard interface for all of them.
<br>
Here is code example:

```
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-ada-001", n=2, best_of=2)
llm("Tell me a joke")

Output:
>>> '\n\nWhy did the chicken cross the road?\n\nTo get to the other side.'
```
Let's see how to pass multiple inputs:

```
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"]*15)
len(llm_result.generations)

Output:
>>> 30
```
As llm_result.generations return list, it can be accessed like standard list.

If LLM provider provide usage data, LangChain will return that too but it will not work for all LLMs so consider it not standardized. As our usage is focused on OpenAI, it will work.

```
llm_result.llm_output
```
Results will look like:
```
{'token_usage': {'completion_tokens': 3903,
  'total_tokens': 4023,
  'prompt_tokens': 120}}
```

Further, you can also count number of tokens in query:

```
llm.get_num_tokens("what a joke")
>>> 3
```